<a href="https://colab.research.google.com/github/Varchala/Secure_Private_AI/blob/main/pate_varchala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #, force_remount = True)

Mounted at /content/drive


In [1]:
import keras
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import l2
from keras.callbacks import Callback, LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras import utils as np_utils
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils

In [2]:
import tensorflow as tf

In [4]:
def extract_n_classes( data, labels,n, cat_y = True):
    data_cl=[]
    y=[]
    # print(data.shape)
    for i in range(n):
            # print(data[ np.argwhere( labels == i ).reshape(-1) ].shape)
            # print(labels.shape)
            # print(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1)].shape,data.shape)
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))

    # print(np.array(data_cl).shape)
    x = np.vstack( (data_cl) )
    y = np.array(y)
    # print("In extract classes function")
    # print(x.shape,y.shape)
    return x, y

# Every teacher has instance of only one class

# 10 Teachers

In [5]:
n_instances = 500
n_teachers = 10

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

x_train, y_train = extract_n_classes( x_train, y_train ,n=100,cat_y=False)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )

In [6]:
teacher_data_x.shape, teacher_data_y.shape

((10, 500, 32, 32, 3), (10, 500, 100))

In [7]:
student_data_x.shape,student_data_y.shape

((500, 32, 32, 3), (500, 100))

In [8]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ],run_eagerly=True)

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    model.save( '/content/drive/MyDrive/SPAI_projectphase2/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.06809999793767929
teacher 1
test accuracy: 0.0568000003695488
teacher 2
test accuracy: 0.05570000037550926
teacher 3
test accuracy: 0.0608999989926815
teacher 4
test accuracy: 0.05550000071525574
teacher 5
test accuracy: 0.0560000017285347
teacher 6
test accuracy: 0.057500001043081284
teacher 7
test accuracy: 0.0575999990105629
teacher 8
test accuracy: 0.0706000030040741
teacher 9
test accuracy: 0.060100000351667404


## training the student model

In [9]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [10]:
# student_data_x

In [11]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [12]:
labels.shape

(10, 500, 100)

In [13]:
student_data_x.shape

(500, 32, 32, 3)

In [14]:
len(student_data_x)

500

In [15]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

In [16]:
student_data_x.shape,student_data_y.shape,votes.shape

((500, 32, 32, 3), (500, 100), (500, 100))

In [17]:
ll = np.argmax( votes, axis=1 )

In [18]:
ll.shape

(500,)

In [19]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((500, 100), (500, 100))

In [20]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.024000000208616257


# 20 Teachers

In [21]:
n_instances = 500 
n_teachers = 20

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

x_train, y_train = extract_n_classes( x_train, y_train ,n=100,cat_y=False)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )

In [22]:
teacher_data_x.shape, teacher_data_y.shape

((20, 500, 32, 32, 3), (20, 500, 100))

In [23]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    model.save( '/content/drive/MyDrive/SPAI_projectphase2/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.06509999930858612
teacher 1
test accuracy: 0.07100000232458115
teacher 2
test accuracy: 0.05790000036358833
teacher 3
test accuracy: 0.06650000065565109
teacher 4
test accuracy: 0.05350000038743019
teacher 5
test accuracy: 0.0658000037074089
teacher 6
test accuracy: 0.059700001031160355
teacher 7
test accuracy: 0.06369999796152115
teacher 8
test accuracy: 0.06129999831318855
teacher 9
test accuracy: 0.05400000140070915
teacher 10
test accuracy: 0.065700002014637
teacher 11
test accuracy: 0.05550000071525574
teacher 12
test accuracy: 0.05660000070929527
teacher 13
test accuracy: 0.06719999760389328
teacher 14
test accuracy: 0.0632999986410141
teacher 15
test accuracy: 0.06260000169277191
teacher 16
test accuracy: 0.047600001096725464
teacher 17
test accuracy: 0.05739999935030937
teacher 18
test accuracy: 0.06520000100135803
teacher 19
test accuracy: 0.06780000030994415


## training the student model

In [24]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [25]:
labels.shape

(20, 500, 100)

In [26]:
student_data_x.shape

(500, 32, 32, 3)

In [27]:
len(student_data_x)

500

In [28]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

In [29]:
student_data_x.shape,student_data_y.shape,votes.shape

((500, 32, 32, 3), (500, 100), (500, 100))

In [30]:
ll = np.argmax( votes, axis=1 )

In [31]:
ll.shape

(500,)

In [32]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((500, 100), (500, 100))

In [33]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.02199999988079071


# 50 Teachers

In [34]:
n_instances = 500 
n_teachers = 50

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

x_train, y_train = extract_n_classes( x_train, y_train ,n=100,cat_y=False)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )

In [35]:
teacher_data_x.shape, teacher_data_y.shape

((50, 500, 32, 32, 3), (50, 500, 100))

In [36]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    model.save( '/content/drive/MyDrive/SPAI_projectphase2/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.06279999762773514
teacher 1
test accuracy: 0.05900000035762787
teacher 2
test accuracy: 0.04919999837875366
teacher 3
test accuracy: 0.058800000697374344
teacher 4
test accuracy: 0.05810000002384186
teacher 5
test accuracy: 0.06210000067949295
teacher 6
test accuracy: 0.06769999861717224
teacher 7
test accuracy: 0.05649999901652336
teacher 8
test accuracy: 0.05849999934434891
teacher 9
test accuracy: 0.06620000302791595
teacher 10
test accuracy: 0.06030000001192093
teacher 11
test accuracy: 0.06310000270605087
teacher 12
test accuracy: 0.055799998342990875
teacher 13
test accuracy: 0.05480000004172325
teacher 14
test accuracy: 0.05689999833703041
teacher 15
test accuracy: 0.06069999933242798
teacher 16
test accuracy: 0.05999999865889549
teacher 17
test accuracy: 0.06759999692440033
teacher 18
test accuracy: 0.053700000047683716
teacher 19
test accuracy: 0.05920000001788139
teacher 20
test accuracy: 0.06310000270605087
teacher 21
test accuracy: 0.0625
teacher 

## training the student model

In [37]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [38]:
labels.shape

(50, 500, 100)

In [39]:
student_data_x.shape

(500, 32, 32, 3)

In [40]:
len(student_data_x)

500

In [41]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

In [42]:
student_data_x.shape,student_data_y.shape,votes.shape

((500, 32, 32, 3), (500, 100), (500, 100))

In [43]:
ll = np.argmax( votes, axis=1 )

In [44]:
ll.shape

(500,)

In [45]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((500, 100), (500, 100))

In [46]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.05000000074505806


# 100 Teachers

In [33]:
n_instances = 300
n_teachers = 100

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

x_train, y_train = extract_n_classes( x_train, y_train ,n=100,cat_y=False)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )

In [34]:
teacher_data_x.shape, teacher_data_y.shape

((100, 300, 32, 32, 3), (100, 300, 100))

In [7]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    model.save( '/content/drive/MyDrive/SPAI_projectphase2/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.059300001710653305
teacher 1
test accuracy: 0.06610000133514404
teacher 2
test accuracy: 0.06440000236034393
teacher 3
test accuracy: 0.050599999725818634
teacher 4
test accuracy: 0.06939999759197235
teacher 5
test accuracy: 0.05310000106692314
teacher 6
test accuracy: 0.07020000368356705
teacher 7
test accuracy: 0.06509999930858612
teacher 8
test accuracy: 0.05979999899864197
teacher 9
test accuracy: 0.05959999933838844
teacher 10
test accuracy: 0.06870000064373016
teacher 11
test accuracy: 0.06459999829530716
teacher 12
test accuracy: 0.06809999793767929
teacher 13
test accuracy: 0.049300000071525574
teacher 14
test accuracy: 0.05950000137090683
teacher 15
test accuracy: 0.05739999935030937
teacher 16
test accuracy: 0.06300000101327896
teacher 17
test accuracy: 0.06379999965429306
teacher 18
test accuracy: 0.053199999034404755
teacher 19
test accuracy: 0.06499999761581421
teacher 20
test accuracy: 0.05609999969601631
teacher 21
test accuracy: 0.052600000053

## training the student model

In [10]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [11]:
labels.shape

(100, 300, 100)

In [12]:
student_data_x.shape

(300, 32, 32, 3)

In [13]:
len(student_data_x)

300

In [14]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

In [15]:
student_data_x.shape,student_data_y.shape,votes.shape

((300, 32, 32, 3), (300, 100), (300, 100))

In [16]:
ll = np.argmax( votes, axis=1 )

In [ ]:
ll.shape

In [18]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((300, 100), (300, 100))

In [19]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.06333333253860474


# Every teacher has instances of every class:

# 10 Teachers

In [69]:
n_instances = 500
n_teachers = 10

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )

In [70]:
teacher_data_x.shape, teacher_data_y.shape

((10, 500, 32, 32, 3), (10, 500, 100))

In [71]:
student_data_x.shape,student_data_y.shape

((500, 32, 32, 3), (500, 100))

In [72]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ],run_eagerly=True)

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    model.save( '/content/drive/MyDrive/SPAI_projectphase2/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.06350000202655792
teacher 1
test accuracy: 0.07129999995231628
teacher 2
test accuracy: 0.06679999828338623
teacher 3
test accuracy: 0.058800000697374344
teacher 4
test accuracy: 0.07419999688863754
teacher 5
test accuracy: 0.06520000100135803
teacher 6
test accuracy: 0.06129999831318855
teacher 7
test accuracy: 0.054999999701976776
teacher 8
test accuracy: 0.04610000178217888
teacher 9
test accuracy: 0.05829999968409538


## training the student model

In [73]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [ ]:
# student_data_x

In [76]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [77]:
labels.shape

(10, 500, 100)

In [79]:
student_data_x.shape

(500, 32, 32, 3)

In [80]:
len(student_data_x)

500

In [81]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

In [82]:
student_data_x.shape,student_data_y.shape,votes.shape

((500, 32, 32, 3), (500, 100), (500, 100))

In [83]:
ll = np.argmax( votes, axis=1 )

In [84]:
ll.shape

(500,)

In [85]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((500, 100), (500, 100))

In [86]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.009999999776482582


# 20 Teachers

In [87]:
n_instances = 500 
n_teachers = 20

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )

In [88]:
teacher_data_x.shape, teacher_data_y.shape

((20, 500, 32, 32, 3), (20, 500, 100))

In [89]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    model.save( '/content/drive/MyDrive/SPAI_projectphase2/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.05220000073313713
teacher 1
test accuracy: 0.05829999968409538
teacher 2
test accuracy: 0.04540000110864639
teacher 3
test accuracy: 0.06939999759197235
teacher 4
test accuracy: 0.05609999969601631
teacher 5
test accuracy: 0.05550000071525574
teacher 6
test accuracy: 0.0714000016450882
teacher 7
test accuracy: 0.04969999939203262
teacher 8
test accuracy: 0.0568000003695488
teacher 9
test accuracy: 0.05460000038146973
teacher 10
test accuracy: 0.0723000019788742
teacher 11
test accuracy: 0.056299999356269836
teacher 12
test accuracy: 0.06080000102519989
teacher 13
test accuracy: 0.06800000369548798
teacher 14
test accuracy: 0.06159999966621399
teacher 15
test accuracy: 0.05249999836087227
teacher 16
test accuracy: 0.05810000002384186
teacher 17
test accuracy: 0.06629999727010727
teacher 18
test accuracy: 0.0568000003695488
teacher 19
test accuracy: 0.05429999902844429


## training the student model

In [90]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [91]:
labels.shape

(20, 500, 100)

In [92]:
student_data_x.shape

(500, 32, 32, 3)

In [93]:
len(student_data_x)

500

In [94]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

In [95]:
student_data_x.shape,student_data_y.shape,votes.shape

((500, 32, 32, 3), (500, 100), (500, 100))

In [96]:
ll = np.argmax( votes, axis=1 )

In [97]:
ll.shape

(500,)

In [98]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((500, 100), (500, 100))

In [99]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.014000000432133675


# 50 Teachers

In [20]:
n_instances = 500 
n_teachers = 50

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )

In [21]:
teacher_data_x.shape, teacher_data_y.shape

((50, 500, 32, 32, 3), (50, 500, 100))

In [22]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    model.save( '/content/drive/MyDrive/SPAI_projectphase2/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.06769999861717224
teacher 1
test accuracy: 0.06300000101327896
teacher 2
test accuracy: 0.05620000138878822
teacher 3
test accuracy: 0.06360000371932983
teacher 4
test accuracy: 0.05920000001788139
teacher 5
test accuracy: 0.06689999997615814
teacher 6
test accuracy: 0.052799999713897705
teacher 7
test accuracy: 0.0649000033736229
teacher 8
test accuracy: 0.056299999356269836
teacher 9
test accuracy: 0.05640000104904175
teacher 10
test accuracy: 0.06790000200271606
teacher 11
test accuracy: 0.0568000003695488
teacher 12
test accuracy: 0.0502999983727932
teacher 13
test accuracy: 0.06499999761581421
teacher 14
test accuracy: 0.06750000268220901
teacher 15
test accuracy: 0.06719999760389328
teacher 16
test accuracy: 0.048500001430511475
teacher 17
test accuracy: 0.05640000104904175
teacher 18
test accuracy: 0.053700000047683716
teacher 19
test accuracy: 0.06120000034570694
teacher 20
test accuracy: 0.06400000303983688
teacher 21
test accuracy: 0.060400001704692

## training the student model

In [23]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [24]:
labels.shape

(50, 500, 100)

In [25]:
student_data_x.shape

(500, 32, 32, 3)

In [26]:
len(student_data_x)

500

In [27]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

In [28]:
student_data_x.shape,student_data_y.shape,votes.shape

((500, 32, 32, 3), (500, 100), (500, 100))

In [29]:
ll = np.argmax( votes, axis=1 )

In [30]:
ll.shape

(500,)

In [31]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((500, 100), (500, 100))

In [32]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.057999998331069946


# 100 Teachers

In [39]:
n_instances = 300 
n_teachers = 100

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )

In [40]:
teacher_data_x.shape, teacher_data_y.shape

((100, 300, 32, 32, 3), (100, 300, 100))

In [41]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    model.save( '/content/drive/MyDrive/SPAI_projectphase2/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.039400000125169754
teacher 1
test accuracy: 0.03999999910593033
teacher 2
test accuracy: 0.0333000011742115
teacher 3
test accuracy: 0.05220000073313713
teacher 4
test accuracy: 0.0502999983727932
teacher 5
test accuracy: 0.04800000041723251
teacher 6
test accuracy: 0.04390000179409981
teacher 7
test accuracy: 0.048900000751018524
teacher 8
test accuracy: 0.05000000074505806
teacher 9
test accuracy: 0.04190000146627426
teacher 10
test accuracy: 0.03849999979138374
teacher 11
test accuracy: 0.04659999907016754
teacher 12
test accuracy: 0.04470000043511391
teacher 13
test accuracy: 0.048700001090765
teacher 14
test accuracy: 0.045899998396635056
teacher 15
test accuracy: 0.050999999046325684
teacher 16
test accuracy: 0.05820000171661377
teacher 17
test accuracy: 0.03759999945759773
teacher 18
test accuracy: 0.04780000075697899
teacher 19
test accuracy: 0.03689999878406525
teacher 20
test accuracy: 0.05000000074505806
teacher 21
test accuracy: 0.0416000001132488

## training the student model

In [42]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [43]:
labels.shape

(100, 300, 100)

In [44]:
student_data_x.shape

(300, 32, 32, 3)

In [45]:
len(student_data_x)

300

In [46]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

In [47]:
student_data_x.shape,student_data_y.shape,votes.shape

((300, 32, 32, 3), (300, 100), (300, 100))

In [48]:
ll = np.argmax( votes, axis=1 )

In [49]:
ll.shape

(300,)

In [50]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((300, 100), (300, 100))

In [51]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.05000000074505806
